In [ ]:
import pandas as pd
import plotly.graph_objects as go
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import torch
import warnings

if '+cu' not in torch.__version__:
    warnings.warn("PyTorch is not using CUDA. This may impact performance. To install pytorch with CUDA support run `pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu118`")

In [ ]:
class AnomalyResult:
    def __init__(self, forecast_data, actual_data):
        """
        Initialize AnomalyResult with forecast and actual data
        
        Args:
            forecast_data (pd.DataFrame): DataFrame containing forecast data (mean, floor, hat)
            actual_data (pd.DataFrame): DataFrame containing actual values
        """
        self._forecast = forecast_data
        self._actual = actual_data.copy()
        # Make the underlying numpy arrays immutable
        self._forecast.values.flags.writeable = False
        self._actual.values.flags.writeable = False
    
    @property
    def forecast(self):
        """Read-only access to forecast data"""
        return self._forecast.copy()

    @property
    def actual(self):
        """Read-only access to actual data"""
        return self._actual.copy()

In [ ]:
def generate_forecast(df, prediction_length, verbosity=0):
    data = TimeSeriesDataFrame(df)

    predictor = TimeSeriesPredictor(
        prediction_length=prediction_length,
        quantile_levels=[0.1, 0.5, 0.9],
        target="value",
        verbosity=verbosity
    ).fit(
        data,
        presets="bolt_tiny"  # Use Chronos-Bolt in zero-shot mode
    )

    forecasts = predictor.predict(data)
    return forecasts

In [4]:
def get_minutes_between_dates(from_time_str, to_time_str):
    from_time = pd.Timestamp(from_time_str)
    to_time = pd.Timestamp(to_time_str)
    
    minutes = (to_time - from_time).total_seconds() / 60
    return int(minutes)

In [ ]:
def get_anomaly_summary(df, fromTime, toTime):
    interval_length = 30

    """
    Generate time series forecast using AutoGluon TimeSeriesPredictor.
    
    Args:
        df: DataFrame with time series data, must have 'timestamp' and 'value' columns
        prediction_length: Number of future time steps to predict
        verbosity: Level of output verbosity (0 for silent)
    
    Returns:
        DataFrame with forecasted values
        
    Raises:
        ValueError: If input validation fails
    """

    # Validate DataFrame structure
    required_column = {'value'}
    if not all(col in df.columns for col in required_column):
        raise ValueError(f"DataFrame must contain column: {required_column}")
    
    # Validate data is not empty
    if df.empty:
        raise ValueError("DataFrame cannot be empty")
        
    # Validate data types
    if not pd.api.types.is_numeric_dtype(df['value']):
        raise ValueError("'value' column must contain numeric data")
    
    # Validate DataFrame has DatetimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame must have a DatetimeIndex")

    prediction_steps = get_minutes_between_dates(fromTime, toTime)
    
    prediction_intervals = int(prediction_steps/interval_length)
    dfHistoricalData = df[(df.index < fromTime)]

    dfMean = dfHistoricalData.resample(f'{interval_length}min').mean()
    dfMean['timestamp'] = dfMean.index
    dfMean['item_id'] = 'single_item'

    dfMin = dfHistoricalData.resample(f'{interval_length}min').min()
    dfMin['timestamp'] = dfMin.index
    dfMin['item_id'] = 'single_item'

    dfMax = dfHistoricalData.resample(f'{interval_length}min').max()
    dfMax['timestamp'] = dfMax.index
    dfMax['item_id'] = 'single_item'

    forecastMean = generate_forecast(dfMean, prediction_intervals)
    forecastMin = generate_forecast(dfMin, prediction_intervals)
    forecastMax = generate_forecast(dfMax, prediction_intervals)

    merged = pd.merge(forecastMean[['mean']], forecastMin[['0.1']], 
                     left_index=True, right_index=True, 
                     how='inner')
    
    merged = pd.merge(merged, forecastMax[['0.9']], 
                     left_index=True, right_index=True, 
                     how='inner')
    
    merged = merged.reset_index().set_index('timestamp'). \
        drop(['item_id'], axis=1). \
        resample('1min').interpolate(method='time'). \
        rename(columns={'0.1': 'floor', '0.9': 'hat'})

    merged = pd.merge(merged, df[['value']], 
                     left_index=True, right_index=True, 
                     how='inner'). \
        rename(columns={'value': 'actual'})

    merged['deviation'] = merged.apply(
        lambda x: ((-x['floor'] / x['actual']) + 1 if x['floor'] > x['actual']
            else (x['actual']/x['hat']) - 1 if x['actual'] > x['hat']
            else None), axis = 1
    )

    return AnomalyResult(forecast_data=merged, actual_data=df)

In [ ]:
def plot_deviation(anomalyResult: AnomalyResult):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=anomalyResult.forecast.index, y=anomalyResult.forecast['deviation'], mode='markers', name='Deviation'))

    fig.update_layout(
        title='Deviation',
        xaxis_title='Date',
        yaxis_title='Value',
        template='plotly_white',
        xaxis=dict(
            range=[anomalyResult.forecast.index.min(), anomalyResult.forecast.index.max()]  # Set the x-axis range
        )
    )

    fig.show()


In [ ]:
def plot_prediction(anomalyResult: AnomalyResult):
    # Get the last timestamp from the DataFrame
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=anomalyResult.forecast.index, y=anomalyResult.forecast['mean'], mode='lines', name='Forecast'))
    fig.add_trace(go.Scatter(x=anomalyResult.forecast.index, y=anomalyResult.forecast['floor'], fill=None, line=dict(color='rgba(0,100,80,0.2)'), name=None))
    fig.add_trace(go.Scatter(x=anomalyResult.forecast.index, y=anomalyResult.forecast['hat'], fill='tonexty', line=dict(color='rgba(0,100,80,0.1)'), name='Confidence interval'))

    fig.add_trace(go.Scatter(x=anomalyResult.actual.index, y=anomalyResult.actual['value'], mode='lines', name='Actual'))


    fig.update_layout(
        title='Forecast',
        xaxis_title='Date',
        yaxis_title='Value',
        template='plotly_white',
        xaxis=dict(
            range=[anomalyResult.forecast.index.min(), anomalyResult.forecast.index.max()]  # Set the x-axis range
        )
    )

    fig.show()

In [ ]:
from plotly.subplots import make_subplots

def plot_combined_graphs(anomalyResult: AnomalyResult):
    # Create figure with secondary y-axis
    fig = make_subplots(rows=2, cols=1,
                       shared_xaxes=True,  # This links the x-axes
                       vertical_spacing=0.1)

    # Add prediction plot
    fig.add_trace(
        go.Scatter(x=anomalyResult.forecast.index, 
                  y=anomalyResult.forecast['mean'], 
                  mode='lines', 
                  name='Forecast'),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=anomalyResult.forecast.index, 
                  y=anomalyResult.forecast['floor'], 
                  fill=None, 
                  line=dict(color='rgba(0,100,80,0.2)'), 
                  name='Lower bound'),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=anomalyResult.forecast.index, 
                  y=anomalyResult.forecast['hat'], 
                  fill='tonexty', 
                  line=dict(color='rgba(0,100,80,0.1)'), 
                  name='Upper bound'),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=anomalyResult.actual.index, 
                  y=anomalyResult.actual['value'], 
                  mode='lines', 
                  name='Actual'),
        row=1, col=1
    )

    # Add deviation plot
    fig.add_trace(
        go.Scatter(x=anomalyResult.forecast.index, 
                  y=anomalyResult.forecast['deviation'], 
                  mode='markers', 
                  name='Deviation'),
        row=2, col=1
    )

    # Set the date range for zoom
    xaxis_range = [anomalyResult.forecast.index.min(), anomalyResult.forecast.index.max()
    ]

    # Update layout
    fig.update_layout(
        height=800,  # Increase overall height to accommodate both plots
        title_text="Forecast and Deviation Analysis",
        showlegend=True,
        template='plotly_white',
        xaxis=dict(range=xaxis_range),
        xaxis2=dict(range=xaxis_range)
    )

    # Update y-axes labels
    fig.update_yaxes(title_text="Value", row=1, col=1)
    fig.update_yaxes(title_text="Deviation", row=2, col=1)

    fig.show()